# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`CHEM-Data`*

---
*@author: Evan*\
*@date: 2023-05-17*

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from functools import reduce

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.io.shapereader import Reader

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [3]:
chem = xr.open_dataset('F:/Data/Project_anqing/April/add_data/COMBINE_ACONC_CN3AH_135X138_2023_chem.nc')
grid = xr.open_dataset('F:/Data/Project_anqing/GRIDCRO2D_2023021.nc')

In [4]:
times=pd.date_range('2023-04-06-00','2023-04-10-23',freq='h')
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50

In [5]:
# 结合地形高度HT与各气压层离地高度ZH获得各层实际海拔高度
ht=np.squeeze(grid.HT)
height=np.zeros_like(chem.ZH)
for t in range(np.size(chem.ZH,0)):
    for l in range(np.size(chem.ZH,1)):
        height[t,l,:,:]=chem.ZH[t,l,:,:]+ht

In [6]:
days=1 # set spin-up days
dataset=xr.Dataset(
    data_vars=dict(
        # ! vars from CMAQ
        CO=(['time','level','y','x'],chem.CO[days*24-8:-8,:,:,:].data*30/22.4,{'long name':'Carbon Monoxide','units':'ug m-3'}),
        O3=(['time','level','y','x'],chem.O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Ozone','units':'ug m-3'}),
        NO=(['time','level','y','x'],chem.NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Nitric Oxide','units':'ug m-3'}),
        NO2=(['time','level','y','x'],chem.NO2[days*24-8:-8,:,:,:].data*37/22.4,{'long name':'Nitrogen Dioxide','units':'ug m-3'}),
        SO2=(['time','level','y','x'],chem.SO2[days*24-8:-8,:,:,:].data*64/22.4,{'long name':'Sulfur Dioxide','units':'ug m-3'}),
        VOC=(['time','level','y','x'],chem.VOC[days*24-8:-8,:,:,:].data,{'long name':'Volitile Organic Compounds','units':'ppbC'}),
        PM25=(['time','level','y','x'],chem.PM25_TOT[days*24-8:-8,:,:,:].data,{'long name':'','units':'ug m-3'}),
        PM10=(['time','level','y','x'],chem.PM10[days*24-8:-8,:,:,:].data,{'long name':'','units':'ug m-3'}),
        # ! wwind
        wwind=(['time','level','y','x'],chem.WWind[days*24-8:-8,:,:,:].data,{'long name':'Vertical Wind Speed','units':'m s-1'}),
        # ! altitude
        HT=(['time','level','y','x'],height[days*24-8:-8,:,:,:].data,{'long name':'Altitudes','units':'m'}),
        ),
    coords=dict(
        time=times,
        level=pres,
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202304',
        grid='CN3AH_135X138',
    ),
)
dataset

<xarray.Dataset>
Dimensions:    (time: 120, level: 38, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-04-06 ... 2023-04-10T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 135.8 101.5 72.89
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    CO         (time, level, y, x) float32 217.8 215.9 209.5 ... 42.96 42.83
    O3         (time, level, y, x) float32 63.65 64.79 ... 1.441e+03 1.452e+03
    NO         (time, level, y, x) float32 0.0003355 0.0001241 ... 3.057e-09
    NO2        (time, level, y, x) float32 6.389 6.387 6.206 ... 0.1228 0.1224
    SO2        (time, level, y, x) float32 8.786 8.675 ... 0.005137 0.005089
    VOC        (time, level, y, x) float32 30.01 29.11 27.98 ... 0.8586 0.8488
    PM25       (time, level, y, x) float32 18.15 17.82 17.2 ... 0.2306 0.2298
    PM10       (time, level, y, x) float32 38.24 37.41 35.99 ... 0.2617 0.2604
    wwind      (time, level, y, x) float32 0.004251 -0.2148 ... -0.001815
    HT         (time, level, y, x) float32 367.5 633.0 ... 1.935e+04 1.934e+04
Attributes:
    case:     Anqing_202304
    grid:     CN3AH_135X138

In [7]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset.data_vars}
dataset.to_netcdf('D:/Download/April_05wind_chem.nc',encoding=encoding)